# N-grams
---
An N-Gram is a connected string of N items from a sample of text or speech. The N-Gram could be comprised of large blocks of words, or smaller sets of syllables. N-Grams are used as the basis for functioning N-Gram models, which are instrumental in natural language processing as a way of predicting upcoming text or speech.
![N-gram example](./images/n-gram.jpg)

## Supporting articles
- [What are n-grams?](https://www.educative.io/answers/what-are-n-grams) 
- [Implement N-Grams using Python NLTK – A Step-By-Step Guide](https://www.askpython.com/python/examples/n-grams-python-nltk)

In [1]:
import pandas as pd
from nltk import ngrams

In [2]:
corpora = pd.read_csv('../../data/english/clean-datasets/combine dataset of measuring hate speech, hate speech offensive, Twitter Tweets Sentiment/clean combined dataset.csv')

In [3]:
corpora.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64598 entries, 0 to 64597
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   raw sentence             64598 non-null  object 
 1   clean sentence training  62756 non-null  object 
 2   label                    62787 non-null  float64
 3   clean sentence EDA       62787 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.0+ MB
